## Demo of Model
### Get Features

Begin by defining the sample:
- 100 positive + 100 negative cases
- for each class, half are without prior attacks, half after prior attacks

In [ ]:
# from extract_features import batch_extract_features
# from torch_helpers import ChangeDetectionDataset
import pandas as pd

DATA_DIR = '../data/'
annotation_path = DATA_DIR + 'annotations_ukraine.csv'
before_dir = DATA_DIR + 'images_ukraine_extracted_before'
after_dir = DATA_DIR + 'images_ukraine_extracted_after'

annotation_df = pd.read_csv(annotation_path)
metadata_df = pd.read_csv(DATA_DIR + 'metadata_ukraine.csv')

# join on timeline_id=id
annotation_df = annotation_df.merge(metadata_df, left_on='timeline_id', right_on='id')

# only clearest rows
annotation_df = annotation_df[
    (annotation_df['before_clear_percent'] == 100) &
    (annotation_df['after_clear_percent'] == 100)
]

# sample
N = 1600

# Filter the DataFrame for the required conditions
cum_attack_0_event_1 = annotation_df[(annotation_df['cum_attack'] == 1) & (annotation_df['event'] == 1)].sample(n=int(N/4), random_state=42)
cum_attack_0_event_0 = annotation_df[(annotation_df['cum_attack'] == 0)].sample(n=int(N/4), random_state=42)
cum_attack_gt_0_event_1 = annotation_df[(annotation_df['cum_attack'] > 0) & (annotation_df['event'] == 1)].sample(n=int(N/4), random_state=42)
cum_attack_gt_0_event_0 = annotation_df[(annotation_df['cum_attack'] > 0) & (annotation_df['event'] == 0)].sample(n=int(N/4), random_state=42)

# Combine the samples into a single DataFrame
sample_df = pd.concat([cum_attack_0_event_1, cum_attack_0_event_0, cum_attack_gt_0_event_1, cum_attack_gt_0_event_0])

# Shuffle the resulting DataFrame (optional)
sample_df = sample_df.sample(frac=1, random_state=42).reset_index(drop=True)
# save sample df
sample_df.to_csv(DATA_DIR + '../data/demo/demo_sample_ukraine.csv', index=False)

sample_ids = sample_df['timeline_id'].tolist()

Then extract features based on embedding encodings from foundation model pretrained on google earth images.

In [ ]:
from extract_features import batch_extract_features

DATA_DIR = '../data/'
annotation_path = DATA_DIR + 'annotations_ukraine.csv'
before_dir = DATA_DIR + 'images_ukraine_extracted_before'
after_dir = DATA_DIR + 'images_ukraine_extracted_after'

batch_extract_features(image_ids=sample_ids, before_dir=before_dir,
              after_dir=after_dir, checkpoint_path=DATA_DIR + "model_weights/vit-b-checkpoint-1599.pth", 
              output_dir=DATA_DIR + "features", device='cuda', use_merged=True)

### Model Functions

In [ ]:
'''
Key Changes and Improvements:
    Memory Efficiency: The code processes one image pair at a time and clears CUDA cache after each processing step to handle your 4GB GPU constraint.

    Pooling Options: I've included multiple pooling methods to experiment with:
        avg: Average pooling of all tokens (except CLS)
        max: Max pooling of tokens
        cls: Using only the CLS token
        all: Concatenating CLS with averaged patch tokens
    
    Sample Limiting: Added a sample_limit parameter to control how many images to process (200 as you specified).

    Visualizations: Added proper evaluation metrics and visualizations including ROC curve, confusion matrix, and feature importance.

    Saving Intermediate Results: The features are saved to avoid recomputation during experimentation
'''


import torch
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import torch.nn.functional as F

# Import functions from your existing module
def process_image_pair_diff_first(row_data, before_dir, after_dir, pooling_method, device):
    """Process a single image pair, first computing differences then pooling"""
    image_id, label = row_data
    # Build paths
    before_path = os.path.join(before_dir, str(image_id) + '.npz')
    after_path = os.path.join(after_dir, str(image_id) + '.npz')
    
    # Skip if files don't exist
    if not os.path.exists(before_path) or not os.path.exists(after_path):
        return None
        
    try:
        # Load features
        before_data = np.load(before_path)
        after_data = np.load(after_path)
        before_features = before_data['features']
        after_features = after_data['features']
        
        # Convert to torch
        before_features_torch = torch.tensor(before_features)
        after_features_torch = torch.tensor(after_features)
        
        # Calculate token-level differences first
        diff_features_torch = after_features_torch - before_features_torch
        
        # Now apply pooling to the difference features
        if pooling_method == 'avg':
            # Average pool all tokens except CLS
            pooled_diff = diff_features_torch[:, 1:, :].mean(dim=1).numpy()
        elif pooling_method == 'max':
            # Max pool all tokens except CLS
            pooled_diff = diff_features_torch[:, 1:, :].max(dim=1)[0].numpy()
        elif pooling_method == 'cls':
            # Use only the CLS token difference
            pooled_diff = diff_features_torch[:, 0, :].numpy()
        elif pooling_method == 'all':
            # Concatenate CLS difference with average of patch token differences
            cls_diff = diff_features_torch[:, 0, :].numpy()
            avg_diff = diff_features_torch[:, 1:, :].mean(dim=1).numpy()
            pooled_diff = np.concatenate([cls_diff, avg_diff], axis=1)
        
        return (pooled_diff.flatten(), label, image_id)
    
    except Exception as e:
        print(f"Error processing image {image_id}: {e}")
        return None

def process_image_pair(row_data, before_dir, after_dir, pooling_method, device):
    """Process a single image pair in parallel"""
    image_id, label = row_data
    # Build paths
    before_path = os.path.join(before_dir, str(image_id) + '.npz')
    after_path = os.path.join(after_dir, str(image_id) + '.npz')
    
    # Skip if files don't exist
    if not os.path.exists(before_path) or not os.path.exists(after_path):
        return None
        
    try:
        # Load features (using numpy directly to avoid PyTorch overhead)
        before_data = np.load(before_path)
        after_data = np.load(after_path)
        before_features = before_data['features']
        after_features = after_data['features']
        
        # Convert to torch for pooling
        before_features_torch = torch.tensor(before_features)
        after_features_torch = torch.tensor(after_features)
        
        # Apply pooling
        if pooling_method == 'avg':
            pooled_before = before_features_torch[:, 1:, :].mean(dim=1).numpy()
            pooled_after = after_features_torch[:, 1:, :].mean(dim=1).numpy()
        elif pooling_method == 'max':
            pooled_before = before_features_torch[:, 1:, :].max(dim=1)[0].numpy()
            pooled_after = after_features_torch[:, 1:, :].max(dim=1)[0].numpy()
        elif pooling_method == 'cls':
            pooled_before = before_features_torch[:, 0, :].numpy()
            pooled_after = after_features_torch[:, 0, :].numpy()
        elif pooling_method == 'all':
            cls_before = before_features_torch[:, 0, :].numpy()
            avg_before = before_features_torch[:, 1:, :].mean(dim=1).numpy()
            pooled_before = np.concatenate([cls_before, avg_before], axis=1)
            
            cls_after = after_features_torch[:, 0, :].numpy()
            avg_after = after_features_torch[:, 1:, :].mean(dim=1).numpy()
            pooled_after = np.concatenate([cls_after, avg_after], axis=1)
        
        # Calculate difference
        diff_features = pooled_after - pooled_before
        
        return (diff_features.flatten(), label, image_id)
    
    except Exception as e:
        print(f"Error processing image {image_id}: {e}")
        return None

def create_change_detection_dataset(before_dir, after_dir, label_path, diff_first, 
                               pooling_method, sample_limit=5000, device='cuda'):
    """
    Create a dataset for change detection
    
    Args:
        before_dir: Directory with before images
        after_dir: Directory with after images
        label_path: Path to CSV with labels
        pooling_method: Method for pooling features
        sample_limit: Maximum number of samples to process
        device: Device to run on
        
    Returns:
        X_diff: Feature differences (properly shaped for model training)
        y: Labels
    """
    """
    Create a dataset for change detection with parallel processing
    """
    import multiprocessing
    from concurrent.futures import ProcessPoolExecutor
    from functools import partial
    # Load labels
    labels_df = pd.read_csv(label_path)
    
    # Limit samples if needed
    if sample_limit and sample_limit < len(labels_df):
        labels_df = labels_df.sample(sample_limit, random_state=42)
    
    # Create a list of (image_id, label) tuples
    image_data = list(zip(labels_df['timeline_id'], labels_df['event']))
    

    if diff_first:
        # Create partial function with fixed arguments
        process_func = partial(
            process_image_pair_diff_first, 
            before_dir=before_dir, 
            after_dir=after_dir, 
            pooling_method=pooling_method, 
            device=device
        )
        n_workers = 8
        # Use process pool to handle the processing
        with ProcessPoolExecutor(max_workers=n_workers) as executor:
            results = list(tqdm(executor.map(process_func, image_data), total=len(image_data)))
        
    else:
        # Create partial function with fixed arguments
        process_func = partial(
            process_image_pair, 
            before_dir=before_dir, 
            after_dir=after_dir, 
            pooling_method=pooling_method, 
            device=device
        )
        n_workers = 8
        # Use process pool to handle the processing
        with ProcessPoolExecutor(max_workers=n_workers) as executor:
            results = list(tqdm(executor.map(process_func, image_data), total=len(image_data)))
        
    # Filter out None results and separate features, labels, image_ids
    valid_results = [r for r in results if r is not None]
    X_diff, y, image_ids = zip(*valid_results) if valid_results else ([], [], [])
    
    # Convert to numpy arrays
    X_diff = np.array(X_diff)
    y = np.array(y)
    
    print(f"Final X_diff shape: {X_diff.shape}")
    
    return X_diff, y, image_ids

def train_lasso_logistic_regression(X_train, y_train, C):
    """
    Train a Lasso logistic regression model
    
    Args:
        X_train: Training features
        y_train: Training labels
        C: Regularization parameter (1/lambda)
        
    Returns:
        Trained model
    """
    # Create and train model
    model = LogisticRegression(
        penalty='l1',
        solver='liblinear',
        C=C,
        max_iter=100000,
        random_state=42
    )
    model.fit(X_train, y_train)
    
    return model

def evaluate_model(model, X_test, y_test):
    """
    Evaluate a trained model
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test labels
        
    Returns:
        Dictionary of evaluation metrics
    """
    # Get predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
    }
    
    # ROC AUC
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    metrics['auc'] = auc(fpr, tpr)
    metrics['fpr'] = fpr
    metrics['tpr'] = tpr
    
    return metrics, y_pred

def visualize_results(model, metrics, X_test, y_test, image_ids_test=None):
    """
    Visualize model results
    
    Args:
        model: Trained model
        metrics: Metrics dictionary
        X_test: Test features
        y_test: Test labels
        image_ids_test: List of image IDs for the test set
    """
    # 1. Plot ROC curve
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(metrics['fpr'], metrics['tpr'], 'b-', label=f'AUC = {metrics["auc"]:.3f}')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='lower right')
    
    # 2. Plot confusion matrix
    plt.subplot(1, 2, 2)
    cm = metrics['confusion_matrix']
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['No Change', 'Change'])
    plt.yticks(tick_marks, ['No Change', 'Change'])
    
    # Add text annotations
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     horizontalalignment="center",
                     color="grey" if cm[i, j] > thresh else "red")
    
    plt.tight_layout()
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    
    plt.tight_layout()
    plt.show()
    
    # 3. Feature importance
    if hasattr(model, 'coef_'):
        top_n = 20
        coef = model.coef_[0]
        
        # Get indices of features with highest absolute weights
        top_indices = np.argsort(np.abs(coef))[-top_n:]
        
        plt.figure(figsize=(10, 6))
        plt.barh(range(top_n), coef[top_indices])
        plt.yticks(range(top_n), [f'Feature {i}' for i in top_indices])
        plt.xlabel('Weight')
        plt.ylabel('Feature')
        plt.title(f'Top {top_n} Feature Weights')
        plt.tight_layout()
        plt.show()

def run_change_detection_demo(pooling_method, C, diff_first,
                              from_pooling=False, sample_limit=5000):
    """Complete change detection demo pipeline"""
    # Paths
    before_dir = '../data/features/before'
    after_dir = '../data/features/after'
    label_path = '../data/demo/demo_sample_ukraine.csv'
    
    # Parameters    
    if not from_pooling:
        # 1. Create dataset
        print("Creating dataset...")
        X_diff, y, image_ids = create_change_detection_dataset(
            before_dir=before_dir,
            after_dir=after_dir,
            label_path=label_path,
            pooling_method=pooling_method,
            sample_limit=sample_limit,
            diff_first=diff_first
        )
        
        print(f"Dataset created with {len(X_diff)} samples")
        print(f"Feature vector shape: {X_diff.shape}")
        
        # 2. Save features to avoid recomputation (optional)
        np.save('../data/demo/change_detection_features.npy', X_diff)
        np.save('../data/demo/change_detection_labels.npy', y)
        np.save('../data/demo/change_detection_image_ids.npy', np.array(image_ids))

    if from_pooling:
        # Load features
        X_diff = np.load('../data/demo/change_detection_features.npy')
        y = np.load('../data/demo/change_detection_labels.npy')
        image_ids = np.load('../data/demo/change_detection_image_ids.npy')
        print(f"Loaded dataset with {len(X_diff)} samples")
        print(f"Feature vector shape: {X_diff.shape}")    
    
    # 3. Split data
    X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
        X_diff, y, image_ids, test_size=0.2, random_state=42, stratify=y
    )
    
    # 4. Train model
    print("Training Lasso Logistic Regression model...")
    model = train_lasso_logistic_regression(X_train, y_train, C=C)
    
    # save model
    import joblib
    # set model name
    model_name = f'../data/demo/lasso_logistic_model_pool={pooling_method}_n={sample_limit}.pkl'
    joblib.dump(model, model_name)

    # 5. Evaluate model
    print("Evaluating model...")
    metrics, y_pred = evaluate_model(model, X_test, y_test)
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print(f"AUC: {metrics['auc']:.4f}")
    
    # 6. Visualize results
    visualize_results(model, metrics, X_test, y_test, ids_test)
    
    return model, metrics, X_diff, y, image_ids, y_pred, ids_test

# # Run the demo
# if __name__ == "__main__":
#     model, X_diff, y, image_ids = run_change_detection_demo()

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# Import the demo function
def run_grid_search(sample_limit=5000):
    """
    Run a grid search over different hyperparameters:
    - pooling_method: 'avg', 'max', 'cls', 'all'
    - C: 0.1 to 0.7 in 0.1 steps
    - diff_first: True, False
    
    Optimize by running feature extraction once per (pooling_method, diff_first) combination
    """
    # Define parameters for grid search
    pooling_methods = ['avg', 'max', 'cls', 'all']
    C_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    diff_first_values = [True, False]
    
    # Create a dataframe to store results
    results = []
    
    # Create directory for results if it doesn't exist
    os.makedirs('../data/demo/grid_search', exist_ok=True)
    
    # Run grid search
    for pooling_method in pooling_methods:
        for diff_first in diff_first_values:
            print(f"\n{'='*80}")
            print(f"Testing pooling_method={pooling_method}, diff_first={diff_first}")
            print(f"{'='*80}")
            
            # First run: Extract features (from_pooling=False)
            print(f"Extracting features with pooling_method={pooling_method}, diff_first={diff_first}...")
            start_time = time.time()
            
            # Use C=0.1 for the initial run, but we'll reuse the features for all C values
            model, metrics, X_diff, y, image_ids = run_change_detection_demo(
                pooling_method=pooling_method,
                C=C_values[0],
                diff_first=diff_first,
                from_pooling=False,
                sample_limit=sample_limit
            )
            
            # Save this configuration's results
            end_time = time.time()
            results.append({
                'pooling_method': pooling_method,
                'diff_first': diff_first,
                'C': C_values[0],
                'precision': metrics['precision'],
                'recall': metrics['recall'],
                'f1': metrics['f1'],
                'auc': metrics['auc'],
                'accuracy': metrics['accuracy'],
                'time': end_time - start_time
            })
            
            # Now iterate through the remaining C values using the saved features
            for C in C_values[1:]:
                print(f"Testing C={C} with pooling_method={pooling_method}, diff_first={diff_first}...")
                start_time = time.time()
                
                model, metrics, X_diff, y, image_ids = run_change_detection_demo(
                    pooling_method=pooling_method,
                    C=C,
                    diff_first=diff_first,
                    from_pooling=True,  # Reuse the extracted features
                    sample_limit=sample_limit
                )
                
                # Save this configuration's results
                end_time = time.time()
                results.append({
                    'pooling_method': pooling_method,
                    'diff_first': diff_first,
                    'C': C,
                    'precision': metrics['precision'],
                    'recall': metrics['recall'],
                    'f1': metrics['f1'],
                    'auc': metrics['auc'],
                    'accuracy': metrics['accuracy'],
                    'time': end_time - start_time
                })
                
            # Save intermediate results after each (pooling_method, diff_first) combination
            results_df = pd.DataFrame(results)
            results_df.to_csv(f'../data/demo/grid_search/results_intermediate_{pooling_method}_{diff_first}.csv', index=False)
    
    # Convert to dataframe and save
    results_df = pd.DataFrame(results)
    results_df.to_csv('../data/demo/grid_search/grid_search_results.csv', index=False)
    
    # Find best configuration
    best_row = results_df.loc[results_df['precision'].idxmax()]
    print("\nBest configuration:")
    print(f"Pooling Method: {best_row['pooling_method']}")
    print(f"Diff First: {best_row['diff_first']}")
    print(f"C: {best_row['C']}")
    print(f"Precision: {best_row['precision']:.4f}")
    
    return results_df

if __name__ == "__main__":
    # Run the grid search with 800 samples (modify as needed)
    results = run_grid_search(sample_limit=1600)
    
    # Print a summary of the top 5 configurations by precision
    print(b"\nTop 5 configurations by precision:")
    top_results = results.sort_values('precision', ascending=False).head(5)
    print(top_results[['pooling_method', 'diff_first', 'C', 'precision']])

In [ ]:
_, _, _, _, _, y_pred, ids_test = run_change_detection_demo(pooling_method='max',C=0.2, diff_first=True,
                          from_pooling=False, sample_limit=50000)

In [ ]:
# turn ids test and ypred into df
predicted = pd.DataFrame(data={"timeline_id":ids_test, "pred":y_pred})

# load sample csv
sample_df = pd.read_csv("/run/media/hennes/SSD_2TB/Dropbox/projects/seminar_paper/data/demo/demo_sample_ukraine.csv")


In [ ]:
# merge sample_df and predicted on timeline_id
merged = sample_df.merge(right=predicted, how='right', on='timeline_id')

In [ ]:
merged['correct'] = (merged['event'] == merged['pred']).astype(int)

In [ ]:
merged.columns

In [ ]:
# group merged by correct and compute mean of cum_attack and show table of most common locations
merged.groupby('correct').agg({"cum_attack":'mean'})

In [ ]:
merged[['first_attack', 'event']].value_counts()

In [ ]:
# create dummy variable that is 1 if cum_attack is 1 and 0 otherwise
merged['first_attack'] = merged['cum_attack'].apply(lambda x: 1 if x == 1 else 0)
merged.groupby('first_attack').agg({'correct':'mean'})